In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 40.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=27cd84ca83c67687f2ba534bc5726d7393ee1da6837fc2429cbaaa73f6212460
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col,when

session = SparkSession.builder.master("local").appName("FirstApp").getOrCreate()

In [ ]:
d1_df = session.read.csv('dataset1.csv', header=True, inferSchema=True)
d2_df = session.read.csv('dataset2.csv', header=True, inferSchema=True)

d1_df.show(5)
d2_df.show(5)

+-----+-----------+---+---------+---------+--------------+
|   id|        age|sex|weight_kg|height_cm|    medication|
+-----+-----------+---+---------+---------+--------------+
|  469|57.05276775|  F|     64.0|    155.5|Anticonvulsant|
| 8724| 75.7412248|  F|     78.0|    162.0| No-medication|
| 6736|    70.7789|  M|     73.0|    170.5| No-medication|
|24180|78.24717531|  F|     60.0|    148.0| No-medication|
|17072| 54.1918766|  M|     55.0|    161.0| No-medication|
+-----+-----------+---+---------+---------+--------------+
only showing top 5 rows

+-----+-----------+------------+----------+
|   id|   fracture|waiting_time|       bmd|
+-----+-----------+------------+----------+
|  469|no fracture|          18|    0.8793|
| 8724|no fracture|          56|0.79460001|
| 6736|no fracture|          10|0.90670002|
|24180|no fracture|          14|    0.7112|
|17072|no fracture|          20|0.79089999|
+-----+-----------+------------+----------+
only showing top 5 rows



In [ ]:


d1_df_clean = d1_df.dropna('any')
d2_df_clean = d2_df.dropna('any')
d1_df_clean.count()
d2_df_clean.count()

169

In [ ]:
d2_df_clean.describe('bmd').show()

+-------+-------------------+
|summary|                bmd|
+-------+-------------------+
|  count|                169|
|   mean| 0.7831041407692312|
| stddev|0.16652888156774745|
|    min|         0.40759999|
|    max|          1.3624001|
+-------+-------------------+



In [ ]:
d1_df_clean.select('sex').distinct().show(3)
d1_df_clean.select('medication').distinct().show(3)


+---+
|sex|
+---+
|  F|
|  M|
+---+

+---------------+
|     medication|
+---------------+
|Glucocorticoids|
|  No-medication|
| Anticonvulsant|
+---------------+



In [ ]:
d1_df_clean = d1_df_clean.withColumn('sex', when(col('sex') == 'F', 0)
                                        .when(col('sex') == 'M', 1)
                                        .otherwise(2)
)

d1_df_clean = d1_df_clean.withColumn('medication', when(col('medication') == 'Glucocorticoids', 1)
                                        .when(col('medication') == 'No-medication', 2)
                                        .when(col('medication') == 'Anticonvulsant', 3)
                                        .otherwise(4)
)

d1_df_clean.show(3)



+----+-----------+---+---------+---------+----------+
|  id|        age|sex|weight_kg|height_cm|medication|
+----+-----------+---+---------+---------+----------+
| 469|57.05276775|  0|     64.0|    155.5|         3|
|8724| 75.7412248|  0|     78.0|    162.0|         2|
|6736|    70.7789|  1|     73.0|    170.5|         2|
+----+-----------+---+---------+---------+----------+
only showing top 3 rows



In [ ]:


d1_m = d1_df_clean.withColumn("height_m", d1_df_clean.height_cm/100)

d1_m.show(3)

d1_m_bmi = d1_m.withColumn("bmi", d1_m.weight_kg/(d1_m.height_m ** 2))

d1_m_bmi.show(3)





+----+-----------+---+---------+---------+----------+--------+
|  id|        age|sex|weight_kg|height_cm|medication|height_m|
+----+-----------+---+---------+---------+----------+--------+
| 469|57.05276775|  0|     64.0|    155.5|         3|   1.555|
|8724| 75.7412248|  0|     78.0|    162.0|         2|    1.62|
|6736|    70.7789|  1|     73.0|    170.5|         2|   1.705|
+----+-----------+---+---------+---------+----------+--------+
only showing top 3 rows

+----+-----------+---+---------+---------+----------+--------+------------------+
|  id|        age|sex|weight_kg|height_cm|medication|height_m|               bmi|
+----+-----------+---+---------+---------+----------+--------+------------------+
| 469|57.05276775|  0|     64.0|    155.5|         3|   1.555|  26.4678818457212|
|8724| 75.7412248|  0|     78.0|    162.0|         2|    1.62|29.721079103795148|
|6736|    70.7789|  1|     73.0|    170.5|         2|   1.705|25.111583147719745|
+----+-----------+---+---------+---------+

In [ ]:


new_dataset = d1_m_bmi.join(d2_df_clean, d2_df.id == d1_m_bmi.id).select('age', 'sex', 'medication', 'bmi', 'bmd')

new_dataset.show(3)




+-----------+---+----------+------------------+----------+
|        age|sex|medication|               bmi|       bmd|
+-----------+---+----------+------------------+----------+
|57.05276775|  0|         3|  26.4678818457212|    0.8793|
| 75.7412248|  0|         2|29.721079103795148|0.79460001|
|    70.7789|  1|         2|25.111583147719745|0.90670002|
+-----------+---+----------+------------------+----------+
only showing top 3 rows



In [ ]:
features = new_dataset.drop('bmd').columns

assembler = VectorAssembler( inputCols=features, outputCol="Myfeatures")

final_dataset= assembler.transform(new_dataset).select('Myfeatures','bmd')

final_dataset.show(5)

+--------------------+----------+
|          Myfeatures|       bmd|
+--------------------+----------+
|[57.05276775,0.0,...|    0.8793|
|[75.7412248,0.0,2...|0.79460001|
|[70.7789,1.0,2.0,...|0.90670002|
|[78.24717531,0.0,...|    0.7112|
|[54.1918766,1.0,2...|0.79089999|
+--------------------+----------+
only showing top 5 rows



In [ ]:
train_df,test_df = final_dataset.randomSplit([0.8, 0.2])
train_df.show(5)
test_df.show(5)

+--------------------+----------+
|          Myfeatures|       bmd|
+--------------------+----------+
|[35.81405763,1.0,...|    0.7895|
|[36.64831103,1.0,...|0.85600001|
|[37.46199496,1.0,...|0.68839997|
|[40.23236742,1.0,...|     1.039|
|[43.45071208,1.0,...| 1.0197999|
+--------------------+----------+
only showing top 5 rows

+--------------------+----------+
|          Myfeatures|       bmd|
+--------------------+----------+
|[39.25494707,1.0,...|    1.0502|
|[41.35119985,1.0,...|    0.7076|
|[41.41761507,1.0,...|    1.0003|
|[42.81703317,0.0,...|0.88880002|
|[44.23967859,1.0,...|    0.9824|
+--------------------+----------+
only showing top 5 rows



In [ ]:
lin_reg = LinearRegression(featuresCol = 'Myfeatures', labelCol='bmd',)
linear_model = lin_reg.fit(train_df)

In [ ]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [-0.0033586476165329833,0.0902557495784229,-0.002165919317263794,0.015665092992662853]
Intercept: 0.5648785742676364


In [ ]:
predictions = linear_model.transform(test_df)
predictions.select("prediction","bmd","Myfeatures").show(10)

+------------------+----------+--------------------+
|        prediction|       bmd|          Myfeatures|
+------------------+----------+--------------------+
|0.9144599739579358|    1.0502|[39.25494707,1.0,...|
|0.7890104586932023|    0.7076|[41.35119985,1.0,...|
|0.9155609897912849|    1.0003|[41.41761507,1.0,...|
| 0.801479204672471|0.88880002|[42.81703317,0.0,...|
|0.9072323257705502|    0.9824|[44.23967859,1.0,...|
|0.9047191432818517|    0.9716|[48.28948823,1.0,...|
|0.7362409919811056|    0.6904|[48.52365084,0.0,...|
|1.0133831278058718|0.89950001|[48.81687141,1.0,...|
|0.7147480951281903|0.73989999|[50.27605888,0.0,...|
|0.7827430938854405|    0.6904|[51.04837034,0.0,...|
+------------------+----------+--------------------+
only showing top 10 rows



In [ ]:
new_patient = session.sql("select 56 as age, 0 as sex, 3 as medication, 26 as bmi")
new_patient.show()
new_patient=assembler.transform(new_patient).select('Myfeatures')
new_patient.show()
predictions = linear_model.transform(new_patient)
predictions.select("Myfeatures","prediction").show()

+---+---+----------+---+
|age|sex|medication|bmi|
+---+---+----------+---+
| 56|  0|         3| 26|
+---+---+----------+---+

+-------------------+
|         Myfeatures|
+-------------------+
|[56.0,0.0,3.0,26.0]|
+-------------------+

+-------------------+------------------+
|         Myfeatures|        prediction|
+-------------------+------------------+
|[56.0,0.0,3.0,26.0]|0.7775889675992322|
+-------------------+------------------+

